In [55]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
input_text = au.read_txt_file_lines()

Actually reminded me of an example from Fluent Python. Will try to implement something like that. (https://learning.oreilly.com/library/view/fluent-python/9781491946237/ch01.html)

In [70]:
hand = collections.namedtuple('hand', ['cards', 'bet'])

## create ranking by mapping all possible combinations to a rank
all_cards_sorted_part1 = 'AKQJT98765432'
dict_ranking_part1 = {}
rank = 0 
for c in itertools.product(all_cards_sorted_part1[::-1], repeat=5):  # reverse so we can start with rank=0
    dict_ranking_part1[''.join(c)] = rank
    rank += 1
assert 13 ** 5 < 1e6  # check that we have less than 1e6 combinations
## to determine overall rank, just add combis as 1e6 intervals. bit lazy but works 
dict_combi_to_overall_rank = {'five of a kind': 6e6, 'four of a kind': 5e6, 'full house': 4e6, 
                              'three of a kind': 3e6, 'two pairs': 2e6, 'one pair': 1e6, 'high card': 0}

class Cards:
    def __init__(self, cards, dict_ranking):
        self.cards = cards
        assert type(cards) == str
        assert len(cards) == 5 
        self.rank = dict_ranking[cards]
        self.combi = self.determine_combi()
        self.determine_overall_rank()
        
    def determine_overall_rank(self):
        self.overall_rank = int(dict_combi_to_overall_rank[self.combi] + self.rank)

    def determine_combi(self, cards=None):
        ## get unique cards
        if cards is None:
            cards = self.cards
        unique_cards = set(cards)
        if len(unique_cards) == 1:
            return 'five of a kind'
        elif len(unique_cards) == 2:
            ## check if we have 4 of a kind or full house
            for c in unique_cards:
                if cards.count(c) == 4:
                    return 'four of a kind'
            return 'full house'
        elif len(unique_cards) == 3:
            ## check if we have 3 of a kind or two pairs
            for c in unique_cards:
                if cards.count(c) == 3:
                    return 'three of a kind'
            return 'two pairs'
        elif len(unique_cards) == 4:
            return 'one pair'
        elif len(unique_cards) == 5:
            return 'high card'

    def __repr__(self):  # for debugging
        return f'Cards({self.cards}), combi {self.combi}, rank {self.rank}, overall rank {self.overall_rank}'
    
## load all hands
all_hands = [] 
for l in input_text:
    data = l.split()
    cards = Cards(cards=data[0], dict_ranking=dict_ranking_part1)
    bet = int(data[1])
    all_hands.append(hand(cards, bet))

In [71]:
ii = 0
total_winnings = 0
for k in sorted(all_hands, key=lambda x: x.cards.overall_rank, reverse=False):  # now we can easily sort by overall rank :) 
    ii += 1
    total_winnings += k.bet * ii

print(f'total winnings {total_winnings}')

total winnings 250951660


## part 2
this is now pretty easy, just need to overwrite some methods. will inherit class and feed new ranking

In [85]:
## new ranking for part 2:
all_cards_sorted_part2 = 'AKQT98765432J'
dict_ranking_part2 = {}
rank = 0
for c in itertools.product(all_cards_sorted_part2[::-1], repeat=5):  # reverse so we can start with rank=0
    dict_ranking_part2[''.join(c)] = rank
    rank += 1

class CardsPart2(Cards): # inherit all methods, but write new determine_combi method to replace J
    def __init__(self, cards, dict_ranking):
        super().__init__(cards, dict_ranking)
        self.determine_combi_part2()
        
    def determine_combi_part2(self):
        unique_cards = set(self.cards)
        new_cards = copy.deepcopy(self.cards)
        if 'J' in unique_cards:
            if new_cards == 'JJJJJ': # special case, otherwise max_count will be empty
                pass 
            else:
                counts = []
                for c in unique_cards - set('J'):  # get c with highest count, don't count J
                    counts.append(new_cards.count(c))
                max_count = max(counts)
                for c in 'AKQT98765432': # replace c with J, go from high to low and then break
                    if new_cards.count(c) == max_count:
                        new_cards = new_cards.replace('J', c)
                        break

        ## now rerun combi with new cards, and determine overall rank again:
        self.combi = self.determine_combi(cards=new_cards)
        self.determine_overall_rank()

In [86]:
CardsPart2('AJQJT', dict_ranking_part1)

Cards(AJQJT), combi three of a kind, rank 364320, overall rank 3364320

In [87]:
all_hands = [] 
for l in input_text:
    data = l.split()
    cards = CardsPart2(cards=data[0], dict_ranking=dict_ranking_part2)
    bet = int(data[1])
    all_hands.append(hand(cards, bet))
    
ii = 0
total_winnings = 0
for k in sorted(all_hands, key=lambda x: x.cards.overall_rank, reverse=False):
    # print(k)
    ii += 1
    total_winnings += k.bet * ii

print(f'total winnings {total_winnings}')

total winnings 251481660
